In [1]:
{-# LANGUAGE InstanceSigs #-}

In [2]:
class Monad m => Writer m where
    write :: String -> m ()

In [3]:
newtype W a = W (a, String)

instance Functor W where
    fmap :: (a -> b) -> W a -> W b
    fmap f (W (a, s)) = W (f a, s)

instance Applicative W where
    pure :: a -> W a
    pure a = W (a, "")

    (<*>) :: W (a -> b) -> W a -> W b
    (<*>) (W (f, s)) (W (a, s')) = W (f a, s ++ s')

instance Monad W where
    (>>=) :: W a -> (a -> W b) -> W b
    (>>=) (W (a, s)) f = let W (b, s') = f a in W (b, s ++ s')

instance Writer W where
    write s = W ((), s)

output :: W a -> String
output (W (a, s)) = s

In [4]:
import Control.Monad.Trans.Class

In [5]:
newtype C m a = C ((a -> Action m) -> Action m)

instance Functor (C m) where
    fmap :: (a -> b) -> C m a -> C m b
    fmap f (C a) = C $ \k -> a (k . f)

instance Applicative (C m) where
    pure :: a -> C m a
    pure a = C $ \k -> k a
    
    (<*>) :: C m (a -> b) -> C m a -> C m b
    (<*>) (C f) (C a) = C $ \k -> f $ \f' -> a $ \a' -> k (f' a')

instance Monad (C m) where
    (>>=) :: C m a -> (a -> C m b) -> C m b
    (>>=) (C m) f = C $ \c -> m (\a -> let C m' = (f a) in m' c)

data Action m
    = Atom (m (Action m))
    | Fork (Action m) (Action m)
    | Stop

In [8]:
action :: Monad m => C m a -> Action m
action (C m) = m $ \_ -> Stop

atom :: Monad m => m a -> C m a
atom m = C $ \c -> Atom $ c <$> m

stop :: Monad m => C m a
stop = C $ \_ -> Stop

par :: Monad m => C m a -> C m a -> C m a
par (C m1) (C m2) = C $ \c -> Fork (m1 c) (m2 c)

fork :: Monad m => C m a -> C m ()
fork m = C $ \c -> Fork (action m) (c ())

instance MonadTrans C where
    lift :: Monad m => m a -> C m a
    lift = atom

In [11]:
round :: Monad m => [Action m] -> m ()
round [] = pure ()
round (a:as) = case a of
    Atom am -> do
        a' <- am
        round (as ++ [a'])
    Fork a1 a2 -> round (as ++ [a1, a2])
    Stop -> round as

run :: Monad m => C m a -> m ()
run m = round [action m]